In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/Homework1

Mounted at /gdrive
/gdrive/My Drive/Homework1


In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
import matplotlib.pyplot as plt
from random import randint

In [3]:
seed = 45
import os
import random

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [4]:
def load_data(folder="public_data.npz", resolution=96, head_only=False):
    images = []

    loaded = np.load(folder, allow_pickle=True)

    # Iterate through files in the specified folder
    for i, img in enumerate(loaded['data']):
        # Normalize image pixel values to a float range [0, 1]
        #img = (img / 255).astype(np.float32)

        # Convert image from BGR to RGB
        #img = img[...,::-1]

        # Make the image dataset squared
        dim = min(img.shape[:-1])
        img = img[(img.shape[0]-dim)//2:(img.shape[0]+dim)//2, (img.shape[1]-dim)//2:(img.shape[1]+dim)//2, :]

        # Resize the image to 224x224 pixels
        #img = tfkl.Resizing(224, 224)(img)
        img = tfkl.Resizing(resolution, resolution)(img)

        if img is not None:
            images.append(img)

        if (head_only and i == 9):
           break

    labels = loaded['labels']
    loaded.close()

    if (head_only):
       labels = labels[:10]

    y = LabelEncoder().fit_transform(labels)
    y = tfk.utils.to_categorical(y, 2)

    return np.array(images), y




def display_random_images(X, y, num_img=10):
  # Create subplots for displaying items
  fig, axes = plt.subplots(2, num_img//2, figsize=(20, 9))
  for i in range(num_img):
      image = randint(0, X.shape[0] - 1)

      ax = axes[i%2, i%num_img//2]
      ax.imshow(np.clip(X[image], 0, 255))  # Display clipped item images
      ax.text(0.5, -0.1, str(image) + ' ' + str(y[image]), size=12, ha="center", transform=ax.transAxes)
      ax.axis('off')
  plt.tight_layout()
  plt.show()




def delete_outliers(X, y):
  shrek = 137
  trololo = 5143

  new_X = []
  new_y = []

  num_outliers = 0

  for i, sample in enumerate(X):
    if (not (np.array_equal(sample, X[shrek]) or np.array_equal(sample, X[trololo]))):
      new_X.append(sample)
      new_y.append(y[i])
    else:
      num_outliers += 1

  return np.array(new_X), np.array(new_y), num_outliers

In [5]:
X, y = load_data('public_data.npz')
X, y, num_outliers = delete_outliers(X, y)

In [6]:
# Split data into train_val and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=500, stratify=np.argmax(y,axis=1))

# Further split train_val into train and validation sets
X_train, X_val, y_train_0, y_val = train_test_split(X_train_val, y_train_val, test_size=500, stratify=np.argmax(y_train_val,axis=1))

print(X_train.shape, y_train_0.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(4004, 96, 96, 3) (4004, 2)
(500, 96, 96, 3) (500, 2)
(500, 96, 96, 3) (500, 2)


In [7]:
augment1 = tf.keras.Sequential([
    tfkl.RandomFlip(),
    tfkl.RandomTranslation(height_factor = (-0.3,0.1), width_factor = (-0.1,0.3), fill_mode = 'reflect'),
    tfkl.RandomZoom(0.4, fill_mode = 'reflect'),
    tfkl.RandomBrightness(0.1, value_range=(0,255)),
])

augment2 = tf.keras.Sequential([
    tfkl.RandomFlip(),
    tfkl.RandomZoom(0.2, fill_mode = 'reflect'),
    tfkl.RandomBrightness(0.3, value_range=(0,255)),
    tfkl.RandomRotation((-1,1), fill_mode = 'reflect'),
])

new_X_train_1 = augment1(X_train[np.where((y_train_0[:, 0] == 0) & (y_train_0[:, 1] == 1))])
augmented_X_train_2 = augment2(X_train)
augmented_X_train_1 = augment1(augmented_X_train_2)

X_train = np.append(X_train ,augmented_X_train_2, axis = 0)
X_train = np.append(X_train ,augmented_X_train_1, axis = 0)
X_train = np.append(X_train ,new_X_train_1, axis = 0)

y_train = np.append(y_train_0, y_train_0, axis = 0)
y_train = np.append(y_train, y_train_0, axis = 0)
for k in range(new_X_train_1.shape[0]):
    y_train = np.append(y_train, [[0,1]], axis = 0)

In [8]:
# Define key model parameters
input_shape = X_train.shape[1:]  # Input shape for the model
output_shape = y_train.shape[1]  # Output shape for the model
batch_size = 16                # Batch size for training, always a power of 2!!
epochs = 400

In [9]:
mobile = tfk.applications.ConvNeXtTiny(
    input_shape=(96, 96, 3),
    include_top=False,
    weights="imagenet",
    include_preprocessing=True,
    pooling='avg',
)
mobile.trainable = False

In [10]:
inputs = tfk.Input(shape=input_shape)
# Connect MobileNetV2 to the input
x = mobile(inputs)
x = tfkl.Dropout(rate = 1/5)(x)
x = tfkl.Dense(64)(x)
#x = tfkl.BatchNormalization()(x)
x = tfkl.Activation('relu')(x)
x = tfkl.Dropout(rate = 1/7)(x)
x = tfkl.Dense(64)(x)
#x = tfkl.BatchNormalization()(x)
x = tfkl.Activation('relu')(x)
x = tfkl.Dropout(rate = 1/7)(x)
x = tfkl.Dense(16)(x)
#x = tfkl.BatchNormalization()(x)
x = tfkl.Activation('relu')(x)
x = tfkl.Dropout(rate = 1/7)(x)
# Add a Dense layer with 2 units and softmax activation as the classifier
outputs = tfkl.Dense(2, activation='softmax')(x)

# Create a Model connecting input and output
model_1 = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Compile the model with Categorical Cross-Entropy loss and Adam optimizer
model_1.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.AdamW(learning_rate=4e-3, weight_decay=8e-5), metrics=['accuracy'])

# Display model summary
model_1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 convnext_tiny (Functional)  (None, 768)               27820128  
                                                                 
 dropout (Dropout)           (None, 768)               0         
                                                                 
 dense (Dense)               (None, 64)                49216     
                                                                 
 activation (Activation)     (None, 64)                0         
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 64)                4160  

In [11]:
history = model_1.fit(
    x = X_train, # We need to apply the preprocessing thought for the MobileNetV2 network
    y = y_train,
    batch_size = 16,
    epochs = 400,
    validation_data = (X_val, y_val), # We need to apply the preprocessing thought for the MobileNetV2 network
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=30, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.9, patience = 8, min_lr = 2e-6)
    ]
).history

Epoch 1/400
846/846 [==============================] - 62s 47ms/step - loss: 0.6060 - accuracy: 0.6700 - val_loss: 0.4878 - val_accuracy: 0.7800 - lr: 0.0040
Epoch 2/400
846/846 [==============================] - 38s 45ms/step - loss: 0.5385 - accuracy: 0.7306 - val_loss: 0.4904 - val_accuracy: 0.7400 - lr: 0.0040
Epoch 3/400
846/846 [==============================] - 35s 41ms/step - loss: 0.5148 - accuracy: 0.7499 - val_loss: 0.4927 - val_accuracy: 0.7780 - lr: 0.0040
Epoch 4/400
846/846 [==============================] - 34s 40ms/step - loss: 0.4994 - accuracy: 0.7583 - val_loss: 0.4496 - val_accuracy: 0.7920 - lr: 0.0040
Epoch 5/400
846/846 [==============================] - 33s 40ms/step - loss: 0.4833 - accuracy: 0.7663 - val_loss: 0.4152 - val_accuracy: 0.8120 - lr: 0.0040
Epoch 6/400
846/846 [==============================] - 34s 40ms/step - loss: 0.4772 - accuracy: 0.7744 - val_loss: 0.4401 - val_accuracy: 0.8000 - lr: 0.0040
Epoch 7/400
846/846 [==============================]

In [12]:
# Evaluate the model on the test set
test_accuracy = model_1.evaluate(X_test,y_test,verbose=0)[-1]
print('Test set accuracy %.4f' % test_accuracy)

Test set accuracy 0.8640


In [13]:
model_1.save('convnext_nobatchnorm_NO_INVERSION')

In [ ]:
del model_1

In [18]:
ft_model = tfk.models.load_model('convnext_nobatchnorm_NO_INVERSION')

In [19]:
ft_model.get_layer('convnext_tiny').trainable = True

In [12]:
print(len(ft_model.get_layer('convnext_tiny').layers))

152


In [13]:
ft_model.get_layer('convnext_tiny').summary()

Model: "convnext_tiny"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 96, 96, 3)]          0         []                            
                                                                                                  
 convnext_tiny_prestem_norm  (None, 96, 96, 3)            0         ['input_1[0][0]']             
 alization (Normalization)                                                                        
                                                                                                  
 convnext_tiny_stem (Sequen  (None, 24, 24, 96)           4896      ['convnext_tiny_prestem_normal
 tial)                                                              ization[0][0]']               
                                                                                      

In [20]:
# Freeze first N layers
N = 10    # last block
for i, layer in enumerate(ft_model.get_layer('convnext_tiny').layers[:-N]):
  layer.trainable=False

In [21]:
for i, layer in enumerate(ft_model.get_layer('convnext_tiny').layers):
  print(layer.name, layer.trainable)

input_1 False
convnext_tiny_prestem_normalization False
convnext_tiny_stem False
convnext_tiny_stage_0_block_0_depthwise_conv False
convnext_tiny_stage_0_block_0_layernorm False
convnext_tiny_stage_0_block_0_pointwise_conv_1 False
convnext_tiny_stage_0_block_0_gelu False
convnext_tiny_stage_0_block_0_pointwise_conv_2 False
convnext_tiny_stage_0_block_0_layer_scale False
convnext_tiny_stage_0_block_0_identity False
tf.__operators__.add False
convnext_tiny_stage_0_block_1_depthwise_conv False
convnext_tiny_stage_0_block_1_layernorm False
convnext_tiny_stage_0_block_1_pointwise_conv_1 False
convnext_tiny_stage_0_block_1_gelu False
convnext_tiny_stage_0_block_1_pointwise_conv_2 False
convnext_tiny_stage_0_block_1_layer_scale False
convnext_tiny_stage_0_block_1_identity False
tf.__operators__.add_1 False
convnext_tiny_stage_0_block_2_depthwise_conv False
convnext_tiny_stage_0_block_2_layernorm False
convnext_tiny_stage_0_block_2_pointwise_conv_1 False
convnext_tiny_stage_0_block_2_gelu Fals

In [22]:
ft_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.AdamW(learning_rate=1e-4, weight_decay=3.5e-5), metrics='accuracy')

In [23]:
history = ft_model.fit(
    x = X_train, # We need to apply the preprocessing thought for the MobileNetV2 network
    y = y_train,
    batch_size = 16,
    epochs = 400,
    validation_data = (X_val, y_val), # We need to apply the preprocessing thought for the MobileNetV2 network
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=15, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.9, patience = 8, min_lr = 2e-6)
    ]
).history

Epoch 1/400
846/846 [==============================] - 68s 62ms/step - loss: 0.4823 - accuracy: 0.7976 - val_loss: 0.2528 - val_accuracy: 0.9000 - lr: 1.0000e-04
Epoch 2/400
846/846 [==============================] - 46s 54ms/step - loss: 0.3967 - accuracy: 0.8222 - val_loss: 0.2381 - val_accuracy: 0.9080 - lr: 1.0000e-04
Epoch 3/400
846/846 [==============================] - 44s 52ms/step - loss: 0.3548 - accuracy: 0.8434 - val_loss: 0.2452 - val_accuracy: 0.8920 - lr: 1.0000e-04
Epoch 4/400
846/846 [==============================] - 44s 52ms/step - loss: 0.3252 - accuracy: 0.8568 - val_loss: 0.2451 - val_accuracy: 0.9080 - lr: 1.0000e-04
Epoch 5/400
846/846 [==============================] - 44s 52ms/step - loss: 0.2944 - accuracy: 0.8713 - val_loss: 0.2294 - val_accuracy: 0.8980 - lr: 1.0000e-04
Epoch 6/400
846/846 [==============================] - 44s 52ms/step - loss: 0.2605 - accuracy: 0.8865 - val_loss: 0.2304 - val_accuracy: 0.9160 - lr: 1.0000e-04
Epoch 7/400
846/846 [=======

In [24]:
ft_model.save('convnext_finetuned_nobatchnorm_NO_INVERSION')

In [25]:
ft_model.evaluate(X_test,y_test)

16/16 [==============================] - 5s 163ms/step - loss: 0.2381 - accuracy: 0.9040


[0.23812024295330048, 0.9039999842643738]

In [ ]:
X_test

array([[[[ 74., 124.,  39.],
         [ 54., 103.,  21.],
         [ 59., 105.,  30.],
         ...,
         [128., 148.,  87.],
         [126., 146.,  85.],
         [141., 161., 100.]],

        [[ 79., 129.,  44.],
         [ 61., 110.,  28.],
         [ 60., 104.,  29.],
         ...,
         [136., 156.,  97.],
         [145., 165., 106.],
         [162., 182., 121.]],

        [[ 73., 118.,  37.],
         [ 68., 112.,  33.],
         [ 62., 106.,  31.],
         ...,
         [135., 156., 100.],
         [112., 133.,  77.],
         [157., 178., 121.]],

        ...,

        [[151., 184., 115.],
         [151., 184., 115.],
         [151., 184., 113.],
         ...,
         [153., 184., 125.],
         [145., 176., 119.],
         [150., 181., 124.]],

        [[148., 181., 112.],
         [148., 181., 112.],
         [145., 178., 107.],
         ...,
         [139., 170., 110.],
         [141., 172., 115.],
         [132., 163., 106.]],

        [[142., 175., 106.],
       

In [ ]:
ft_model.save('mobilenetv2_finetuned_NO_INVERSION')